In [81]:
#m Se importan las librerias necesarias
from datetime import datetime
import pandas as pd
import numpy as np
import datetime 
import collections
import pandas_datareader.data as web

In [82]:
# Se importa la librería para importar los historiales de metatraderfrom datetime import datetime
import MetaTrader5 as mt5

In [83]:
local_exe_berlin = 'C:\Program Files\FxPro - MetaTrader 5\terminal64.exe'
local_exe_jorge ='C:\Program Files\MetaTrader 5\terminal64.exe'
# Fechas de descarga
fi = datetime.datetime(2022, 2, 14, 0, 0) #Fecha de inicio de historicos
ff = datetime.datetime(2022, 3, 22, 0, 0)# Fecha final 


La siguiente función será utilizada para poder extraer y leer los historicos de las operaciones de las distintas cuentas en Metratrade5-

In [84]:
def f_leer_archivo(nombre):
    # Numero de posiciones que tuviste debe de ser cambiado
    posiciones = pd.read_csv(nombre,skiprows = 6,nrows = 15)
    for i in range(len(posiciones)):
        posiciones.loc[i,"Price"] = (posiciones.loc[i,"Price"]).replace(" ","")
        posiciones.loc[i,"Price.1"] = (posiciones.loc[i,"Price.1"]).replace(" ","")
        posiciones.loc[i,"Profit"] = (posiciones.loc[i,"Profit"]).replace(" ","")
    posiciones = posiciones.astype({'Price':'float'})
    posiciones = posiciones.astype({'Price.1':'float'})
    posiciones = posiciones.astype({'Profit':'float'})
    #Cambiar nombre de las commodities
    posiciones = posiciones.replace("WTI","WTICOUSD")
    posiciones = posiciones.replace("PALLADIUM","XPDUSD")
    posiciones = posiciones.replace("PLATINUM","XPTUSD")
    return posiciones

In [85]:
def f_leer_pip():
    doc_pips= pd.read_csv("instruments_pips.csv")
    for i in range(len(doc_pips)):
        doc_pips.loc[i,"Instrument"] = (doc_pips.loc[i,"Instrument"]).replace("_","")
    doc_pips = doc_pips.set_index("Instrument")
    new_doc_pips = doc_pips.filter(["TickSize"])
    # Añadir instrumentos que no estan en el excel
    new_doc_pips.loc["BITCOIN"] = .01
    new_doc_pips.loc["ETHEREUM"] = .01
    return new_doc_pips

In [86]:
archivo_pip = f_leer_pip()

In [87]:
def f_pip_size(instrumento: str):
    return 1/archivo_pip.loc[instrumento,"TickSize"]

In [105]:
posiciones = f_leer_archivo("historico_jorge.csv")

In [89]:
def f_columnas_pips(posiciones):
    # Añadir el pipsize de cada instrumento
    pip_size = []
    pip = []
    for i in range(len(posiciones)):
        pip_size.append(f_pip_size(posiciones["Symbol"][i]))
    posiciones["Pip_size"] = pip_size
    # Cantidad de pips resultantes de cada operación 
    for i in range(len(posiciones)):
        if posiciones["Type"][i] == "sell":
            pip.append((posiciones["Price"][i] - posiciones["Price.1"][i])*posiciones["Pip_size"][i])
        else:
            pip.append((posiciones["Price.1"][i] - posiciones["Price"][i])*posiciones["Pip_size"][i])
    posiciones["Pip"] = pip
    posiciones["Pips_acm"] = posiciones["Pip"].cumsum()
    posiciones["Profit_acm"] = posiciones["Profit"].cumsum()
    return posiciones

In [90]:
posiciones

Time  Position    Symbol  Type  Volume        Price  \
0   2022.03.01 02:25:12  49329411  ETHEREUM  sell      10   2934.92000   
1   2022.03.01 15:24:12  49350256    EURUSD   buy      10      1.11441   
2   2022.03.01 15:45:08  49351425   BITCOIN  sell      20  44477.04000   
3   2022.03.03 08:07:11  49432596    EURGBP   buy       1      0.82804   
4   2022.03.03 08:23:08  49432860  WTICOUSD  sell       5    112.39500   
5   2022.03.03 08:37:33  49433058    XPDUSD  sell       1   2681.38000   
6   2022.03.03 08:51:02  49433247    GBPUSD   buy       1      1.34097   
7   2022.03.03 08:57:11  49433357    USDMXN   buy       2     20.64708   
8   2022.03.03 08:59:04  49433389   BITCOIN   buy      10  43500.96000   
9   2022.03.03 09:00:40  49433433    XPTUSD  sell       1   1082.82000   
10  2022.03.03 09:12:15  49433582    USDJPY  sell       1    115.68200   
11  2022.03.03 20:23:53  49467183    EURUSD   buy       4      1.10665   
12  2022.03.03 20:33:29  49467551   BITCOIN  sell       5  42295.13000   
13  2022.03.03 20:34:49  49467605    XPDUSD  sell       2   2772.76000   
14  2022.03.03 20:35:54  49467644    USDJPY  sell       3    115.44100   

        S / L      T / P               Time.1      Price.1  Commission  \
0    2 984.92   2 834.92  2022.03.01 15:06:45   2986.93000         0.0   
1     1.11341    1.11741  2022.03.01 17:10:21      1.11340         0.0   
2   44 777.04  44 177.04  2022.03.01 16:33:16  44173.06000         0.0   
3     0.82604    0.83104  2022.03.04 09:25:12      0.82604         0.0   
4     113.614    109.956  2022.03.03 09:26:06    113.61800         0.0   
5    2 705.60   2 657.16  2022.03.03 09:04:31   2707.34000         0.0   
6     1.33910    1.34237  2022.03.03 09:47:13      1.33908         0.0   
7    20.58859   20.67110  2022.03.03 10:24:18     20.67181         0.0   
8   43 025.18  43 867.18  2022.03.03 15:56:30  43869.41000         0.0   
9    1 093.68   1 071.96  2022.03.03 10:57:49   1093.64000         0.0   
10    115.851    115.574  2022.03.03 17:00:25    115.57400         0.0   
11    1.10353    1.11279  2022.03.04 02:20:00      1.10352         0.0   
12  42 901.99  41 911.52  2022.03.03 22:50:01  41897.92000         0.0   
13   2 819.00   2 741.82  2022.03.04 02:39:36   2819.37000         0.0   
14    115.641    115.349  2022.03.04 02:23:51    115.34800         0.0   

       Swap   Profit  Unnamed: 13  Unnamed: 14  
0      0.00  -520.10          NaN          NaN  
1      0.00 -1010.00          NaN          NaN  
2      0.00  6079.50          NaN          NaN  
3    - 5.35  -266.51          NaN          NaN  
4      0.00 -6115.00          NaN          NaN  
5      0.00 -2596.00          NaN          NaN  
6      0.00  -189.00          NaN          NaN  
7      0.00   239.26          NaN          NaN  
8      0.00  3684.50          NaN          NaN  
9      0.00 -1082.00          NaN          NaN  
10     0.00    93.45          NaN          NaN  
11   - 4.32 -1253.00          NaN          NaN  
12     0.00  1986.05          NaN          NaN  
13  - 24.46 -9322.00          NaN          NaN  
14   - 7.48   241.88          NaN          NaN

In [95]:
def f_estadisticas_ba(posiciones):
    lista_medida = ["Ops totales","Ganadoras","Ganadoras_c","Ganadoras_v","Perdedoras","Perdedoras_c","Perdedoras_v","Mediana(Profit)",
                   "Mediana(Pips)","r_efectividad","r_proporcion","r_efectividad_c","r_efectividad_v"]
    lista_descripcion = ["Operaciones totales","Operaciones ganadoras de compra","Operaciones ganadoras de venta","Operaciones perdedoras de venta",
                        "Mediana de profit de operaciones","Mediana de pips de operaciones","Ganadoras Totales/Operaciones Totales","Ganadoras Totales/Perdedoras Totales",
                         "Ganadoras compras/Operaciones Totales","Ganadoras ventas/Operaciones Totales"]
    lista_descripcion = [1,2,3]
    data = {"valor":[len(posiciones),len(posiciones.loc[posiciones['Profit'] > 0]),len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "buy")])],
           "descripcion": lista_descripcion}
    estadisticas_ba = pd.DataFrame(data)
    return estadisticas_ba

In [96]:
f_estadisticas_ba(posiciones)

valor  descripcion
0     15            1
1      6            2
2      2            3

In [98]:
estadisticas_ba = pd.DataFrame({[len(posiciones),lista_medida[0]],
                                   [len([x for x in posiciones["Profit"].values if x > 0]),lista_medida[1]],
                                   })

NameError: name 'lista_medida' is not defined

In [94]:
len(posiciones.loc[(posiciones['Profit'] > 0) & (posiciones["Type"] == "buy")])

2

In [136]:

posiciones2=posiciones[['Time', "Profit"]]
posiciones2['Time'] = pd.to_datetime(posiciones2['Time']) 
posiciones2.set_index('Time', inplace=True) #index
posiciones3=posiciones2.resample('D').sum()


<ipython-input-136-57f740b96de8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posiciones2['Time'] = pd.to_datetime(posiciones2['Time'])


In [129]:
df1 = pd.DataFrame({"A":[2, 3, 8, 14],  
                   "B":[1, 2, 4, 3],  
                   "C":[5, 3, 9,2]})  
  
print(df1.cumsum(axis = 1))

    A   B   C
0   2   3   8
1   3   5   8
2   8  12  21
3  14  17  19


In [137]:
posiciones3


Profit
Time                
2022-03-01   4549.40
2022-03-02      0.00
2022-03-03 -14578.37

In [ ]:
def f_evolucion_capital(posiciones):
    posiciones2['Time'] = pd.to_datetime(posiciones2['Time']) 
    posiciones.set_index('Time', inplace=True)
    tabla = {"timestamp":[posiciones2.resample('D').sum()],"profit_d":[posiciones.resample('Time', on='Profit').sum()]}
        tabla_evolucion = pd.DataFrame(data)
        return tabla_evolucion
    